# Jupyter Notebook for MicropollDB data analysis

For a start let's import all modules used in the analysis:
- `mysql.connector` lets us connect to a MySql DB hosted on a server
- `pandas` is used for data analysis (e.g. table operations)
- `numpy` ...
- `matplotlib` is for plotting various kind of scientific graphs, with *inline* command to plot figures directly into the notebook itself
- `getpass` is used to request passwords interactively to avoid storing them in the notebook (e.g. for DB user login)

In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import getpass
#pd.set_option('display.max_rows', None)

/home/nibor/.local/share/virtualenvs/MPSchleiSediments-bSFBF4Xj/bin/python


DataTransformerRegistry.enable('default')

Now, connect to the Database:
- first I define a variable called `password`, that receives the output of the function `getpass`
- then connect to the database using the `connect` function of `mysql.connector` and save it into a new variable called `connection`
- the argument for `passwd` calls the content of the variable `password` which in turn calls the `getpass` function, which will open a prompt to type the password on execution

In [2]:
connection = mysql.connector.connect(host = '192.124.245.26',
                                    user = getpass.getpass(),
                                    passwd = getpass.getpass(),
                                    db = 'micropoll')

To check if the connection was established, just print the variable `connection`. It should give something like this:

   ```<mysql.connector.connection.MySQLConnection at 0x..............>```


In [3]:
connection

Let's download the `particles` table using pandas `read_sql_query` function by giving it a SQL `SELECT ...` command together with the second required argument: the variable that contains our open connection to the DB: `connenction`. For better readability I define the SQL command beforehand and call it `query1` 

In [4]:
query1 = '''SELECT distinct
        `p`.`Sample` AS `Sample`,
        `p`.`IDParticles` AS `IDParticles`,
        `s`.`Site_name` AS `Site_name`,
        `s`.`GPS_LON` AS `GPS_LON`,
        `s`.`GPS_LAT` AS `GPS_LAT`,
        `s`.`Compartment` AS `Compartment`,
        `s`.`Contributor` AS `Contributor`,
        `s`.`Project` AS `Project`,
        `p`.`Size_1_[µm]` AS `Size_1_[µm]`,
        `p`.`Size_2_[µm]` AS `Size_2_[µm]`,
        `p`.`Shape` AS `Shape`,
        `p`.`Colour` AS `Colour`,
        `pt`.`polymer_type` AS `polymer_type`,
        `a`.`Library_entry` AS `library_entry`,
        `s`.`Lab_blank` AS `lab_blank_ID`,
        `s`.`IDSample` AS `sample_ID`,
        `s`.`Sampling_weight_[kg]` AS `Sampling_weight_[kg]`,
        `s`.`Fraction_analysed` AS `Fraction_analysed`      
    FROM
        ((((`particles` `p`
        JOIN `samples` `s` ON ((`p`.`Sample` = `s`.`Sample_name`)))
        JOIN `particles2analysis` `pa` ON ((`p`.`IDParticles` = `pa`.`IDParticles`)))
        JOIN `analysis` `a` ON ((`pa`.`IDAnalysis` = `a`.`IDAnalysis`)))
        JOIN `polymer_type` `pt` ON ((`a`.`Result` = `pt`.`IDPolymer_type`)))'''

In [5]:
MP = pd.read_sql_query(query1,connection)

In [6]:
#drop system caused contamination (PTFE, PV23, Parafilm) and certain dyes if they are no distinct indicators for synthetic polymers
MP.drop(MP[(MP.polymer_type == 'Poly (tetrafluoro ethylene)')
                        |(MP.library_entry == 'PV23')
                        |(MP.library_entry == 'Parafilm')
                        |(MP.library_entry == 'PR101')
                        |(MP.library_entry == 'PB15')
                        |(MP.library_entry == 'PW6')
                        |(MP.library_entry == 'PBr29')
                        |(MP.library_entry == 'PY17based')
                        |(MP.library_entry == 'PY74')
                        |(MP.library_entry == 'PB15 + PV23')
                        |(MP.library_entry == 'PV23 + PB15')
                        |(MP.library_entry == 'PB15 + TiO2')
                        |(MP.library_entry == 'PB23 + PY17based')
                        |(MP.library_entry == 'Parafilm/PE')
                        |(MP.library_entry == 'PB15+PY17')
                        |(MP.library_entry == 'PY17+PB15')
                        |(MP.library_entry == 'PV23+PB15+TiO2')
                        |(MP.library_entry == 'PB15+TiO2')
                        |(MP.library_entry == 'TiO2+PB15')
                        |(MP.library_entry == 'PB15+PV23')
                        #|(MP.Sample == 'Blank_20.11.19')
                        #|(MP.Sample == 'Blank_20.11.2019')
                        #|(MP.Sample == 'Blank_20.11.19_IS')
                     #   |(MP.Sample == 'Blank_5.11.19_IS_1')
                      #  |(MP.Sample == 'Blank_5.11.19_IS_2')
                      #  |(MP.Sample == 'Blank_6.11.19_1')
                       # |(MP.Sample == 'Blank_6.11.19_2')
].index, inplace=True)

# Silicon was included here as it is not used during processing

In [7]:
# Replace NaN with 1 in Fraction_analysed, assuming whole sample has been analysed when no value was provided
MP.Fraction_analysed.fillna(1, inplace=True)

In [8]:
# Take 1 divided by Fraction_analysed (rounded to next integer) to get the factor each particle needs to be repeated by to extrapolate to whole sample
# Then do the repetition using np. repeat and write it back into "MP"
MP = MP.loc[np.repeat(MP.index.values,round(1 / MP.Fraction_analysed))]

In [9]:
MP.IDParticles = MP.IDParticles.astype(str) + '_' + MP.groupby('IDParticles').cumcount().astype(str)  # .replace('0','') #--> replace 0 if not wanted

For `particles` I also add a new column by simply addressing a column of it which does not exist yet and fill it the some calculation from 2 other columns (making the *geometric mean* from the size columns `Size_1_[µm]` and `Size_2_[µm]`).

In [10]:
MP['size_geom_mean']=np.sqrt(MP['Size_1_[µm]']*MP['Size_2_[µm]'])
MP.set_index('IDParticles', inplace=True)
MP = MP[MP['Size_1_[µm]']>=50]

I combine certain values, if not needed for further analysis:
- unspecific colours grouped in one ('unspecific')
- shapes other than fibres grouped in one ('irregular')

In [11]:
MP['Colour'].replace(['transparent', 'undetermined', 'white','non-determinable', 'grey', 'brown', 'black'],'unspecific',inplace=True) 
MP['Colour'].replace(['violet'],'blue',inplace=True) 
MP['Shape'].replace(['spherule','irregular','flake','foam','granule','undetermined'],'irregular',inplace=True) #non-fibrous

In [12]:
# take env MP from dataset (without any blinds):
env_MP = MP.loc[(MP.Compartment == 'sediment') & (MP.Site_name == 'Schlei') & (MP.Contributor == 27)].copy()

In [13]:
#take IOW blinds from dataset:
IOW_blind_MP = MP.loc[(MP.Site_name == "lab") & (MP.Project == "BONUS MICROPOLL") & (MP.Contributor == 27)]

blinds_to_use = ['Blank_11.02.19',
                 'Blank_5.11.19_IS_1',
                 'Blank_5.11.19_IS_2',
                 'Blank_6.11.19_1',
                 'Blank_6.11.19_2',
                 'Blank_20.11.19_IS',
                 'Blank_20.11.19']

IOW_blind_MP = IOW_blind_MP.loc[IOW_blind_MP.Sample.isin(blinds_to_use)]

Samples from the environment and from blinds are for now handled together in `samples_MP`.

In [14]:
# append IOW blinds to environmental MP:
samples_MP = pd.concat([env_MP,IOW_blind_MP],axis=0)
samples_MP.sample(2)

,Sample,Site_name,GPS_LON,GPS_LAT,Compartment,Contributor,Project,Size_1_[µm],Size_2_[µm],Shape,Colour,polymer_type,library_entry,lab_blank_ID,sample_ID,Sampling_weight_[kg],Fraction_analysed,size_geom_mean
IDParticles,,,,,,,,,,,,,,,,,,
46174_0,Schlei_S25,Schlei,9.88715,54.6143,sediment,27,BONUS MICROPOLL,615.0,26.0,fibre,unspecific,Poly (ethylene terephthalate),Poly(ethylene terephtalate),1402.0,1215,0.13,1.0,126.451572
45252_1,Schlei_S3,Schlei,9.60065,54.5087,sediment,27,BONUS MICROPOLL,238.0,51.0,irregular,unspecific,Polystyrene,Polystyrene,1400.0,1190,0.06,0.5,110.172592


In [15]:
#take IPF blinds from dataset:
IPF_blank_MP = MP.loc[(MP.sample_ID.isin(samples_MP.lab_blank_ID))].copy()

In [16]:
# For differentiation to env_MP their `size_geom_mean` is renamed to `blank_size_geom_mean`.
IPF_blank_MP.rename(columns={'size_geom_mean':'blank_size_geom_mean'},inplace=True)
IPF_blank_MP['Sample'] = IPF_blank_MP['Sample'].str.replace('Blank_','',1)  # the last option (called count, here "1") was added here because some of the IOW blinds have the sample name "Blank_xxxxx" and their corresponding IPF blanks have the sample name "Blank_Blank_xxxxx". So with count option set to "1", only the first occurence of "Blank_" is replace by "".

### IPF blanks

The IPF blanks should be handled first, so I loop through each blank particle and identify the best matching counterpart in each sample and save its ID for later removal flagging. As this is a bit complicated, see the line by line comments...

In [17]:
samples_MP_copy = samples_MP.copy() #take copy to retain an unaltered version of samples_MP

In [18]:
IPF_elimination_list = pd.DataFrame(columns = [ #create empty dataframe to collect particles-to-be-flagged in a loop
    'ID_blank_particle',
    'ID_sample_particle',
    'Sample',
    'polymer_type',
    'Colour',
    'Shape'])

for label,row_content in IPF_blank_MP.iterrows(): #loop through the list of IPFblanks, particle by particle
    #label contains the particle ID of the current blank particle, row_content contains the data of that particle
    
    current_blank_particle = samples_MP_copy.reset_index().merge(row_content.to_frame().transpose(), on=['Sample','polymer_type','Colour','Shape'], how='inner').set_index('IDParticles')
    #current_blank_particle is basically extract of samples_MP_copy, where only particles which match the current blank particle in all fields after the "on =" are listed
    #where a match is found, all fields of both lines are written as one long line.
    #column names that exist in both of the merged dataframes get an appendix x or y
    #we only need the entry of blank_size_geom_mean to be written as a new column in all lines (i.e. particles) that have the same phenotype as the current blank particle
    #with this we can calculate the difference between the size_geom_mean of the particle and the blank_size_geom_mean 
    
    if len(current_blank_particle) > 0: #there might be the case where no particles were found to match a specific blank particle, so we check for this with this if clause
        
        current_blank_particle['size_diff']=abs((current_blank_particle['size_geom_mean']-current_blank_particle['blank_size_geom_mean'])) #here we take the size difference as described above
        
        eliminee = pd.to_numeric(current_blank_particle['size_diff']).idxmin() #the particle that has the smallest difference in size_geom_mean to that of the current blank particle is our candidate for elimination, and we save its ID as 'eliminee'
        
        IPF_elimination_list = IPF_elimination_list.append(pd.DataFrame({ #now we keep a record entry of all details of the particle that gets eliminated and append it to the prepared data frame
            'ID_blank_particle': label,
            'ID_sample_particle': eliminee,
            'Sample':  current_blank_particle.Sample.iloc[0],
            'polymer_type':  current_blank_particle.polymer_type.iloc[0],
            'Colour':  current_blank_particle.Colour.iloc[0],
            'Shape':  current_blank_particle.Shape.iloc[0]
        }, index=[0]), ignore_index=True)
        
        samples_MP_copy.drop([eliminee],inplace=True) #finally we drop the line of the eliminated particle from our particles dataframe, so we can't match it to another blank particle in the next round
        
        print('For blank particle #',label,': ','Env. particle #',eliminee,'was eliminated.')
    
    else:
        
        print('For blank particle #',label,': ','Nothing to clean up.')

For blank particle # 26681_0 :  Nothing to clean up.
For blank particle # 26684_0 :  Nothing to clean up.
For blank particle # 26686_0 :  Nothing to clean up.
For blank particle # 26689_0 :  Nothing to clean up.
For blank particle # 26690_0 :  Nothing to clean up.
For blank particle # 26691_0 :  Nothing to clean up.
For blank particle # 26692_0 :  Nothing to clean up.
For blank particle # 26694_0 :  Nothing to clean up.
For blank particle # 26709_0 :  Nothing to clean up.
For blank particle # 26710_0 :  Nothing to clean up.
For blank particle # 26712_0 :  Nothing to clean up.
For blank particle # 26723_0 :  Nothing to clean up.
For blank particle # 26725_0 :  Nothing to clean up.
For blank particle # 26728_0 :  Nothing to clean up.
For blank particle # 26733_0 :  Nothing to clean up.
For blank particle # 28292_0 :  Nothing to clean up.
For blank particle # 42143_0 :  Env. particle # 42107_0 was eliminated.
For blank particle # 42158_0 :  Env. particle # 42107_1 was eliminated.
For blan

### IOW blinds
Now I separate the sample particles in blind MP particles as `IOWblind_MP` and environmental MP particles as `env_MP` and form an additional groupby object with them, grouped by sample as `env_MP_samplegrouped`. Note that the input here is the output of the blank particle matching process: `samples_MP_copy`. This is important, because we don't want to eliminate the same particles because of IPF blanks AND IOW blinds. Also, the IOW blind are samples themselves, so there might be particles eliminated because of IPF blanks as well!

In [19]:
IOW_blind_MP = samples_MP_copy[samples_MP_copy.index.isin(IOW_blind_MP.index)].copy()
# For differentiation to env_MP their `size_geom_mean` is renamed to `blind_size_geom_mean`.
IOW_blind_MP.rename(columns={'size_geom_mean':'blind_size_geom_mean'},inplace=True)

env_MP = samples_MP_copy[~samples_MP_copy.index.isin(IOW_blind_MP.index)].copy()

Now I form the blind phenotypes by grouping the particles by `polymer_type`,`Colour` and `Shape`. To know the number of `blinds` we are dealing with I count how many unique entries can be found in the column that shows the sample name.

In [20]:
blind_PhTs = IOW_blind_MP.groupby(['polymer_type','Colour','Shape'])
blinds = pd.unique(IOW_blind_MP.Sample).size

We might have several blinds that replicate each other and together represent the contamination that comes from the process that was used to treat the samples. However, having multiple replicates of blinds, means also a total of X times as many blind particles as there would be correct.

To account for this, we need to combine the blinds into a "synthesised blind" `syn_blind`. To decide which particles are the chosen ones to enter that synthetic blind, we divide them first into their phenotypes, then for each phenotype sort them by size and only select every n<sup>th</sup> particle.

In [21]:
syn_blind = IOW_blind_MP[0:0]

for group_name, group_content in blind_PhTs:
    current_group = group_content.sort_values(by=['blind_size_geom_mean'],ascending=False)
    syn_blind = syn_blind.append(current_group[0::blinds]) #Why is there no inplace option for pandas append?

Now we run a similar procedure as the one above for the IPF blanks, to find out which particles need to be eliminated because a IOW blind particle tell us to. For detailed comments please refer to the inline comments in the IPF blank elimination above.

In [23]:
env_MP_copy = env_MP.copy()
IOW_elimination_list = pd.DataFrame(columns = ['ID_blind_particle','Blind_sample','ID_sample_particle','Sample','polymer_type','Colour','Shape'])

for sample_name, sample_group in env_MP.groupby('Sample'):
    print('Now handling sample: ', sample_name)
    
    for label,row_content in syn_blind.iterrows():
        current_blind_particle = sample_group.reset_index().merge(row_content.to_frame().transpose(), on=['polymer_type','Colour','Shape'], how='inner').set_index('IDParticles')
        
        if len(current_blind_particle) > 0:
            current_blind_particle['size_diff']=abs((current_blind_particle['size_geom_mean']-current_blind_particle['blind_size_geom_mean']))
            eliminee = pd.to_numeric(current_blind_particle['size_diff']).idxmin()
            sample_group.drop([eliminee],inplace=True)
            
            IOW_elimination_list = IOW_elimination_list.append(pd.DataFrame({
            'ID_blind_particle': label,
            'Blind_sample': current_blind_particle.Sample_y.iloc[0],
            'ID_sample_particle': eliminee,
            'Sample':  env_MP_copy.loc[eliminee, 'Sample'],
            'polymer_type':  current_blind_particle.polymer_type.iloc[0],
            'Colour':  current_blind_particle.Colour.iloc[0],
            'Shape':  current_blind_particle.Shape.iloc[0]
             }, index= [0]), ignore_index=True)
            
            env_MP_copy.drop([eliminee],inplace=True)
                    
            #print('For blind particle #',label,': ','Env. particle #',eliminee,'was eliminated.')
        
        #else:
            #print('For blind particle #',label,': ','Nothing to clean up.')
env_MP_copy.to_csv('../csv/env_MP_clean_list_SchleiSediments.csv')

Now handling sample:  Schlei_S10
Now handling sample:  Schlei_S10_15cm
Now handling sample:  Schlei_S11
Now handling sample:  Schlei_S13
Now handling sample:  Schlei_S14
Now handling sample:  Schlei_S15
Now handling sample:  Schlei_S17
Now handling sample:  Schlei_S19
Now handling sample:  Schlei_S1_15cm
Now handling sample:  Schlei_S2
Now handling sample:  Schlei_S22
Now handling sample:  Schlei_S23
Now handling sample:  Schlei_S24
Now handling sample:  Schlei_S25
Now handling sample:  Schlei_S26
Now handling sample:  Schlei_S27
Now handling sample:  Schlei_S3
Now handling sample:  Schlei_S30
Now handling sample:  Schlei_S31
Now handling sample:  Schlei_S5
Now handling sample:  Schlei_S8


For the records we write what has been eliminated to a csv file.

In [24]:
IPF_elimination_list.to_csv('../csv/IPF_elimination_list_SchleiSediments.csv')
IOW_elimination_list.to_csv('../csv/IOW_elimination_list_SchleiSediments.csv')


In [41]:
flagged_particles_IPF_IDParticles = IPF_elimination_list.ID_sample_particle
flagged_particles_IPF_IDblank_particle = IPF_elimination_list.ID_blank_particle

flagged_particles_IOW_IDParticles = IOW_elimination_list.ID_sample_particle
flagged_particles_IOW_IDblank_particle = IOW_elimination_list.ID_blind_particle

flagged_particles = pd.DataFrame({ #create empty dataframe to collect particles-to-be-flagged in a loop
    'IDParticles': flagged_particles_IPF_IDParticles.append(flagged_particles_IOW_IDParticles),
    'IDFlag': np.nan,
    'IDblank_particle': flagged_particles_IPF_IDblank_particle.append(flagged_particles_IOW_IDblank_particle),
    'IDContributor': 27
})

flagged_particles.IDFlag[0:len(flagged_particles_IPF_IDParticles)] = 3
flagged_particles.IDFlag[len(flagged_particles_IPF_IDParticles):] = x

flagged_particles.reset_index(drop = True, inplace = True)

flagged_particles.to_csv('flagged_particles_SchleiSediments.csv', index = False)

<ipython-input-41-3e55984d7416>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flagged_particles.IDFlag[0:len(flagged_particles_IPF_IDParticles)] = 3


NameError: name 'x' is not defined